# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [24]:
import pickle
import re
import sys
import nltk
import pandas as pd
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sqlalchemy import create_engine
from sklearn.metrics import classification_report,accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin


lemmatizer = WordNetLemmatizer()
nltk.download(['wordnet','punkt','stopwords'])
stop_words = nltk.corpus.stopwords.words("english")




[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', con=engine)

In [5]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [6]:
X_train, X_test, y_train, y_test = train_test_split(df['message'], df.iloc[:,4:-1], random_state=444)
print('rows in the original data set: {}'.format(df.shape))
print('rows in the training set: {}'.format(X_train.shape))
print('rows in the test set: {}'.format(X_test.shape))

rows in the original data set: (26216, 40)
rows in the training set: (19662,)
rows in the test set: (6554,)


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    '''this method does the following
    1. normalizing all the words to lower size
    2. removes punctuations
    3. splits the words
    4. removes the stopwords like am,is,have,you,...
    5. lammetizes the words for example running-->run
    '''
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())    # normalize case and remove punctuation
    tokens = word_tokenize(text)    # tokenize text
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]    # lemmatize andremove stop words
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
def model_pipeline(): 
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('multiclf',MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))
    ])
    return pipeline
pipeline = model_pipeline()

In [9]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'multiclf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'multiclf__estimator__bootstrap', 'multiclf__estimator__class_weight', 'multiclf__estimator__criterion', 'multiclf__estimator__max_depth', 'multiclf__estimator__max_features', 'multiclf__estimator__max_leaf_nodes', 'multiclf__estimator__min_impurity_decrease', 'multiclf__estimator__min_impurity_split', 'multiclf__estimator__min_samples_leaf', 'multiclf__estimator__min_samples_split', 'multiclf__estimator__min_weight_fraction_leaf', 'multiclf__estimator__n_estimators', 'multiclf__estimator__n_jobs', 'multiclf__estimator__oob_score', '

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
model = pipeline
model.fit(X_train, y_train);


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred = model.predict(X_test)


In [12]:
for i in range(0,y_test.shape[1]):
    print('------------------------')
    print(y_test.columns[i])
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
    print('%25s accuracy : %.2f' %(y_test.columns[i], accuracy_score(y_test.iloc[:,i], y_pred[:,i])))

------------------------
related
             precision    recall  f1-score   support

          0       0.65      0.48      0.55      1577
          1       0.84      0.92      0.88      4923
          2       0.75      0.17      0.27        54

avg / total       0.79      0.81      0.79      6554

                  related accuracy : 0.81
------------------------
request
             precision    recall  f1-score   support

          0       0.89      0.97      0.93      5440
          1       0.78      0.44      0.56      1114

avg / total       0.87      0.88      0.87      6554

                  request accuracy : 0.88
------------------------
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6522
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6554

                    offer accuracy : 1.00
------------------------
aid_related
             precision    recall  f1-score

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
for i in range(0,y_test.shape[1]):
    print('%25s accuracy : %.2f' %(y_test.columns[i], accuracy_score(y_test.iloc[:,i], y_pred[:,i])))

                  related accuracy : 0.81
                  request accuracy : 0.88
                    offer accuracy : 1.00
              aid_related accuracy : 0.76
             medical_help accuracy : 0.92
         medical_products accuracy : 0.95
        search_and_rescue accuracy : 0.97
                 security accuracy : 0.98
                 military accuracy : 0.97
              child_alone accuracy : 1.00
                    water accuracy : 0.96
                     food accuracy : 0.92
                  shelter accuracy : 0.94
                 clothing accuracy : 0.99
                    money accuracy : 0.98
           missing_people accuracy : 0.99
                 refugees accuracy : 0.97
                    death accuracy : 0.96
                other_aid accuracy : 0.87
   infrastructure_related accuracy : 0.93
                transport accuracy : 0.95
                buildings accuracy : 0.95
              electricity accuracy : 0.98
                    tools accuracy

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
parameters = {'multiclf__estimator__n_estimators': [10,150],
         'multiclf__estimator__max_depth': [30,60]}


cv = 5
model = GridSearchCV(pipeline, parameters, cv=cv)
model.fit(X_train, y_train);

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
y_pred = model.predict(X_test)
for i in range(0,y_test.shape[1]):
    print('------------------------')
    print(y_test.columns[i])
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

------------------------
related
             precision    recall  f1-score   support

          0       0.84      0.09      0.17      1577
          1       0.77      0.99      0.87      4923
          2       0.00      0.00      0.00        54

avg / total       0.78      0.77      0.69      6554

------------------------
request
             precision    recall  f1-score   support

          0       0.87      1.00      0.93      5440
          1       0.92      0.28      0.43      1114

avg / total       0.88      0.87      0.84      6554

------------------------
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6522
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6554

------------------------
aid_related
             precision    recall  f1-score   support

          0       0.75      0.91      0.82      3804
          1       0.82      0.57      0.67      2750

avg / t

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [20]:
class TextLengthExtractor(BaseEstimator, TransformerMixin):
    def text_length(self, text):
        return len(text) - text.count(" ")

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.text_length)
        return pd.DataFrame(X_tagged)

In [25]:
def model_pipeline2():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('text_length', TextLengthExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])

    return pipeline

In [26]:
model = model_pipeline2()
model.fit(X_train, y_train);
y_pred = model.predict(X_test)

In [27]:
for i in range(0,y_test.shape[1]):
    print('------------------------')
    print(y_test.columns[i])
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

------------------------
related
             precision    recall  f1-score   support

          0       0.64      0.56      0.60      1577
          1       0.86      0.91      0.88      4923
          2       0.70      0.13      0.22        54

avg / total       0.81      0.82      0.81      6554

------------------------
request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5440
          1       0.78      0.44      0.57      1114

avg / total       0.88      0.88      0.87      6554

------------------------
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6522
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6554

------------------------
aid_related
             precision    recall  f1-score   support

          0       0.70      0.89      0.78      3804
          1       0.76      0.47      0.58      2750

avg / t

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
for i in range(0,y_test.shape[1]):
    print('%25s accuracy : %.2f' %(y_test.columns[i], accuracy_score(y_test.iloc[:,i], y_pred[:,i])))

                  related accuracy : 0.82
                  request accuracy : 0.88
                    offer accuracy : 1.00
              aid_related accuracy : 0.71
             medical_help accuracy : 0.92
         medical_products accuracy : 0.95
        search_and_rescue accuracy : 0.97
                 security accuracy : 0.98
                 military accuracy : 0.97
              child_alone accuracy : 1.00
                    water accuracy : 0.95
                     food accuracy : 0.92
                  shelter accuracy : 0.93
                 clothing accuracy : 0.99
                    money accuracy : 0.98
           missing_people accuracy : 0.99
                 refugees accuracy : 0.97
                    death accuracy : 0.96
                other_aid accuracy : 0.87
   infrastructure_related accuracy : 0.93
                transport accuracy : 0.95
                buildings accuracy : 0.95
              electricity accuracy : 0.98
                    tools accuracy

### 9. Export your model as a pickle file

In [29]:
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.